In [ ]:
using Interact

In [ ]:
const dot = node(:div, "", style=Dict(:height=>6px, :width=>6px, :borderRadius=>3px))
const on = dot(style(:backgroundColor => :tomato))
const off = dot(style(:backgroundColor => :black))

function showbits(x)
    str = bitstring(x)
    if length(str) % 8 != 0
        extra = 8 - (length(str) % 8)
        str *= "0"^extra
    end
    mx = reshape([str...], 8, div(length(str), 8))
    hbox(mapslices(vbox, map(x -> x=='1' ? on : off, mx), dims=1)...)
end

const boxstyle = Dict(:border=>"1px solid #777")
const redboxstyle = Dict(:border=>"1px solid #f13211")

function showbox(x; bits=true, typ=true, unbox=true, redbox=!isbitstype(typeof(x)))
    if isprimitivetype(typeof(x))
        content= bits ? showbits(x) : x
    elseif unbox > 0 && isbitstype(typeof(x))
        fns = fieldnames(typeof(x))
        content = hbox([showbox(getfield(x, f), typ=!unbox, unbox=unbox) for f in fns])
    elseif fieldcount(typeof(x)) == 0
        content = repr(x)
    else
        fns = fieldnames(typeof(x))
        content = hbox([(unbx = isbitstype(fieldtype(typeof(x), f)); showbox(getfield(x, f), unbox=unbx,
                        typ=!unbox || !unbx, redbox=!unbx)) for f in fns])
    end
    
    if typ
        hbox(
            hbox(
                pad(0.5em, typeof(x)),
                vline(color="#777"),
                pad(0.5em, content)
            )(style=redbox ? redboxstyle : boxstyle)
        )
    else
        pad(0.5em, content)
    end
end

function showbox(x::Array; unbox=true, kwargs...)
    unbox_children = unbox && isbitstype(eltype(x))
    hbox(
        hbox(
            pad(0.5em, typeof(x)),
            vline(color="#777"),
            pad(0.5em, hbox(map(a->showbox(a, typ=!unbox_children, unbox=unbox_children, redbox=!isbitstype(eltype(x))), x)))
        )(style=redboxstyle)
    )
end

## Julia objects in memory

A julia object is a type-value pair

In [ ]:
showbox(42, bits=false)

In [ ]:
showbox(42)

In [ ]:
@manipulate for i = 0:64, fᵢ = Dict("2ⁱ"=>i->2^i, "2ⁱ-1" => i->2^i-1, "-2ⁱ"=>i->-2^i)
    showbox(fᵢ(i))
end

In [ ]:
showbox(UInt32(42))

In [ ]:
showbox('x')

In [ ]:
showbox(UInt128(12345678901234567812345678923456784567))

In [ ]:
showbox(1+2im, unbox=false)

In [ ]:
showbox(0.1f0+0.2f0im, unbox=false)

## Unboxing

Removes redundant type tags, and "inlines" data into a single box!

In [ ]:
showbox(0.1f0+0.2f0im)

In [ ]:
struct Cplx2{T<:Number,S<:Number}
    re::T
    im::S
end

In [ ]:
Cplx2(1,1.0) |> dump

In [ ]:
showbox(Cplx2(1,1.0f0))

**Structs with abstract fields**
- Can't guarrantee the concrete type it can take, hence need to store tag of the field -- means cannot be unboxed
-  require Garbage collection (red box)


In [ ]:
struct Foo
    x::Real
end

In [ ]:
showbox(Foo(1<<27))

In [ ]:
showbox(0.1f0+0.2f0im)

**Unboxing is recursive**

In [ ]:
showbox(42//11+((2^42-1)//8)im, unbox=false) # Not unboxed

In [ ]:
showbox(42//11+((2^42-1)//8)im)

In [ ]:
showbox((1,2.3, 2.3f0))

## Mutable types don't get unboxed!

In [ ]:
mutable struct Cplx4{T<:Real}
    re::T
    im::T
end


Base.:+(a::Cplx4, b::Cplx4) = Cplx4(a.re + b.re, a.im + b.im)
Base.zero(a::Cplx4) = Cplx4(zero(a.re), zero(a.im))

In [ ]:
x = Cplx4(1,0)
x.re = 3
x


In [ ]:
showbox((Cplx4(1,2), Cplx4(3,4)), unbox=true) # Yes, please unbox!! (but it won't)

**Note!!** red boxes must be tracked by the GC! Black ones may be tracked if they have to undergo dispatch at runtime.

One red box makes the parent boxes red! But Julia 1.0 has some optimizations to not track the outer red boxes in the GC under the most common scenarios: e.g. if you're making this tuple in a loop.

## Arrays

In [ ]:
showbox(rand(0:0.1:1, 6), unbox=false)

In [ ]:
showbox(rand(0:0.1:1, 6))

In [ ]:
showbox(Any[1,0.2,3,"four",5])

In [ ]:
showbox([1//2+(3//4)im,
        rand(Int)//rand(Int)+(rand(Int)//rand(Int))im])

In [ ]:
showbox([Cplx4(rand(),rand()), Cplx4(rand(),rand())])

## Strings are immutable only semantically

**but we don't want to specialize structs on their length** so

1. they are tracked by GC
2. structs with strings are tracked by GC

In [ ]:
showbox("hi")

In [ ]:
showbox((255, "hello", 3.3))